In [ ]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
def create_padding_mask(seq: torch.Tensor, pad_idx: int = 0) -> torch.Tensor:
    """パディングマスクを作成する関数。<pad>の部分を1e-9、それ以外を0にする。

    Args:
        seq (torch.Tensor): [batch_size, seq_len]の形状を持つテンソル。入力単語のID列
        pad_idx (int, optional): パディングを表すID。 Defaults to 0.

    Returns:
        torch.Tensor: [batch_size, 1, 1, seq_len] Mult-head Attentionのスコアに加算するためのマスク
    """

    # seq == pad_idx の部分はTrue、それ以外はFalse
    mask = (seq == pad_idx)

    # shapeを [batch_size, 1, 1, seq_len] に変形
    # Trueを1e-9、Falseを0に変換
    # float型に変換しないと加算時にエラーになる
    return mask.unsqueeze(1).unsqueeze(2).float() * -1e9


def create_look_ahead_mask(seq_len: int) -> torch.Tensor:
    """未来の単語を見えなくするための上三角マスクを作成する関数

    Args:
        seq_len (int): シーケンス長

    Returns:
        torch.Tensor: [1, 1, seq_len, seq_len] 対角成分より上が1e-9、それ以外が0の行列
    """
    # torch.triuで上三角行列を取り出す (diagonal=1で対角線のひとつ上から)
    mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1)

    # マスク部分を1e-9、それ以外を0に変換
    return mask.unsqueeze(0).unsqueeze(0).float() * -1e9


def create_masks(src: torch.Tensor, tgt: torch.Tensor, pad_idx: int = 0):
    """EncoderとDecoderに必要なすべてのマスクを一括生成するヘルパー関数

    Args:
        src (torch.Tensor): [batch_size, src_len]
        tgt (torch.Tensor): [batch_size, tgt_len]
        pad_idx (int, optional): パディングID。 Defaults to 0.
    """
    # 1. Encoder用のマスク(Padding Maskのみ)
    src_mask = create_padding_mask(src, pad_idx)

    # 2. Decoder用のマスク(Padding Mask + Look-ahead Mask)
    # 2.1. TargetのPadding Mask [batch_size, 1, 1, tgt_len]
    tgt_pad_mask = create_padding_mask(tgt, pad_idx)

    # 2.2. Look-ahead Mask [1, 1, tgt_len, tgt_len]
    tgt_len = tgt.size(1)
    look_ahead_mask = create_look_ahead_mask(tgt_len).to(tgt.device)

    # 2.3. Padding MaskとLook-ahead Maskを結合 (どちらかが-1e9なら-1e9になるように加算またはmaxを取る)
    # ここで単純に和を取ると-2e9になる箇所ができるが、Softmaxにおいては十分小さいので計算には影響しない
    # 論理和(OR)的にマスクしたいので、最小値を取る実装もよくある(torch.min)
    tgt_mask = torch.min(tgt_pad_mask, look_ahead_mask)

    return src_mask, tgt_mask

In [ ]:
class NoamScheduler:
    def __init__(self, optimizer: torch.optim.Optimizer, d_model: int, warmup_steps: int = 4000):
        """Transformer用学習率スケジューラ (Noam Scheduler)

        Args:
            optimizer (torch.optim.Optimizer): PytorchのOptimizer
            d_model (int): モデルの次元数
            warmup_steps (int, optional): ウォームアップステップ数。 Defaults to 4000.
        """
        self.optimizer = optimizer
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.current_step = 0

    def step(self):
        """1ステップ進め、学習率を更新し、optimizer.step()を実行する
        """
        self.current_step += 1
        lr = self.get_lr()

        # Optimizerの学習率を更新
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        
        self.optimizer.step()
    
    def get_lr(self) -> float:
        """現在のステップ数に基づき学習率を計算

        Returns:
            float: 学習率
        """
        step = self.current_step
        return (self.d_model ** -0.5) * min(step ** -0.5, step * (self.warmup_steps ** -1.5))
    
    def zero_grad(self):
        """optimizerの勾配を初期化(ゼロにする)
        """
        self.optimizer.zero_grad()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_masks():
    src_len = 5
    tgt_len = 5

    # ダミーデータ: 0はpad_idxとする
    # src: [1, 2, 3, 0, 0] 長さ3、pad2
    src = torch.tensor([[1, 2, 3, 0, 0]])
    # tgt: [1, 2, 3, 4, 0] 長さ4、pad1
    tgt = torch.tensor([[1, 2, 3, 4, 0]])

    src_mask, tgt_mask = create_masks(src, tgt, pad_idx=0)

    print(f"Sorce Mask Shape: {src_mask.shape}")
    print(f"Target Mask Shape: {tgt_mask.shape}")

    # グラフ描画
    plt.figure(figsize=(10, 4))

    # Source Mask (Paddingのみ)
    plt.subplot(1, 2, 1)
    # [1, 1, 5] -> [1, 5] 表示のため次元削減
    sns.heatmap(src_mask[0, 0, 0, :].unsqueeze(0).numpy(), cbar=False, square=True)
    plt.title("Source Mask (Padding)")

    # Target Mask (Padding + Look-ahead)
    plt.subplot(1, 2, 2)
    # [1, 5, 5]
    sns.heatmap(tgt_mask[0, 0, :, :].numpy(), cbar=False, square=True)
    plt.title("Target Mask (Padding + Look-ahead)")

    plt.show()

visualize_masks()

In [ ]:
def visualize_scheduler():
    # ダミーモデルとオプティマイザ
    model = torch.nn.Linear(10, 10)
    optimizer = torch.optim.Adam(model.parameters(), lr=0) # 初期lrは0でOK

    d_model = 512
    warmup_steps = 4000
    scheduler = NoamScheduler(optimizer, d_model, warmup_steps)

    lrs = []
    for _ in range(20000):
        scheduler.current_step += 1
        lrs.append(scheduler.get_lr())
    
    plt.plot(lrs)
    plt.xlabel("Step")
    plt.ylabel("Learning Rate")
    plt.title("Noam Scheduler")
    plt.grid(True)
    plt.show()

visualize_scheduler()